# 黑白棋蒙特卡洛树搜索算法实现

## 作者信息
- **课程**: 人工智能
- **作业**: 黑白棋MCTS算法实现
- **日期**: 2025年11月28日


## 1. 算法实现概述

本实验实现了基于蒙特卡洛树搜索（Monte Carlo Tree Search, MCTS）的黑白棋AI算法。主要完成以下核心任务：

### 1.1 实现目标

1. **UCB算法实现**：在MCTS的选择阶段，实现Upper Confidence Bound (UCB)算法来平衡探索与利用
2. **改进的模拟策略**：用启发式策略替代随机模拟，提升搜索效率
3. **完整的MCTS框架**：包括选择（Selection）、扩展（Expansion）、模拟（Simulation）、反向传播（Backpropagation）四个核心步骤

### 1.2 技术要点

- **UCB公式**：平衡已知收益与探索未知节点的权重
- **Roxanne策略**：基于位置优先级的启发式落子策略
- **树节点管理**：维护访问次数、胜利次数等统计信息
- **时间控制**：在限定时间内完成尽可能多的模拟


## 2. 导入必要的库


In [ ]:
from func_timeout import func_timeout, FunctionTimedOut
import datetime
import random
from math import log, sqrt
from time import time
from copy import deepcopy


## 3. 棋盘类实现

棋盘类负责管理8x8的黑白棋棋盘，提供落子、反转、合法性检查等基础功能。


In [ ]:
class ReversiBoard(object):

    def __init__(self):
        """
        初始化棋盘，棋盘大小为8*8，黑棋用 X 表示，白棋用 O 表示，未落子时用 . 表示
        """
        self.board_init()
    
    def board_init(self):
        """
        重置棋盘
        """
        self.empty = '.'  # 未落子状态
        self._board = [[self.empty for _ in range(8)] for _ in range(8)]  # 规格：8*8
        self._board[3][4], self._board[4][3] = 'X', 'X'  # 黑棋棋子初始状态
        self._board[3][3], self._board[4][4] = 'O', 'O'  # 白棋棋子初始状态

    def display(self, step_time=None, total_time=None):  
        """
        打印棋盘
        :param step_time: 每一步的耗时
        :param total_time: 总耗时
        """
        board = self._board
        print(' ', ' '.join(list('ABCDEFGH')))
        for i in range(8):
            print(str(i + 1), ' '.join(board[i]))
            
        if (not step_time) or (not total_time):
            step_time = {"X": 0, "O": 0}
            total_time = {"X": 0, "O": 0}
            print("统计棋局: 棋子总数 / 每一步耗时 / 总时间 ")
            print("黑   棋: " + str(self.count('X')) + ' / ' + str(step_time['X']) + ' / ' + str(total_time['X']))
            print("白   棋: " + str(self.count('O')) + ' / ' + str(step_time['O']) + ' / ' + str(total_time['O']) + '\n')
        else:
            print("统计棋局: 棋子总数 / 每一步耗时 / 总时间 ")
            print("黑   棋: " + str(self.count('X')) + ' / ' + str(step_time['X']) + ' / ' + str(total_time['X']))
            print("白   棋: " + str(self.count('O')) + ' / ' + str(step_time['O']) + ' / ' + str(total_time['O']) + '\n')

    def count(self, color):
        """
        统计 color 一方棋子的数量
        """
        count = 0
        for y in range(8):
            for x in range(8):
                if self._board[x][y] == color:
                    count += 1
        return count

    def get_winner(self):
        """
        判断黑棋和白旗的输赢
        """
        black_count, white_count = 0, 0
        for i in range(8):
            for j in range(8):
                if self._board[i][j] == 'X':
                    black_count += 1
                if self._board[i][j] == 'O':
                    white_count += 1
        if black_count > white_count:
            return 0, black_count - white_count
        elif black_count < white_count:
            return 1, white_count - black_count
        elif black_count == white_count:
            return 2, 0

    def _move(self, action, color):
        """
        落子并获取反转棋子的坐标
        """
        if isinstance(action, str):
            action = self.board_num(action)

        fliped = self._can_fliped(action, color)

        if fliped:
            for flip in fliped:
                x, y = self.board_num(flip)
                self._board[x][y] = color
            x, y = action
            self._board[x][y] = color
            return fliped
        else:
            return False

    def backpropagation(self, action, flipped_pos, color):
        """
        回溯
        """
        if isinstance(action, str):
            action = self.board_num(action)
        self._board[action[0]][action[1]] = self.empty  
        op_color = "O" if color == "X" else "X"
        for p in flipped_pos:
            if isinstance(p, str):
                p = self.board_num(p)
            self._board[p[0]][p[1]] = op_color

    def is_on_board(self, x, y):
        """
        判断坐标是否出界
        """
        return x >= 0 and x <= 7 and y >= 0 and y <= 7

    def _can_fliped(self, action, color):
        """
        检测落子是否合法
        """
        if isinstance(action, str):
            action = self.board_num(action)
        xstart, ystart = action

        if not self.is_on_board(xstart, ystart) or self._board[xstart][ystart] != self.empty:
            return False

        self._board[xstart][ystart] = color
        op_color = "O" if color == "X" else "X"
        flipped_pos = []
        flipped_pos_board = []

        for xdirection, ydirection in [[0, 1], [1, 1], [1, 0], [1, -1], [0, -1], [-1, -1], [-1, 0], [-1, 1]]:
            x, y = xstart, ystart
            x += xdirection
            y += ydirection
            if self.is_on_board(x, y) and self._board[x][y] == op_color:
                x += xdirection
                y += ydirection
                if not self.is_on_board(x, y):
                    continue
                while self._board[x][y] == op_color:
                    x += xdirection
                    y += ydirection
                    if not self.is_on_board(x, y):
                        break
                if not self.is_on_board(x, y):
                    continue
                if self._board[x][y] == color:
                    while True:
                        x -= xdirection
                        y -= ydirection
                        if x == xstart and y == ystart:
                            break
                        flipped_pos.append([x, y])

        self._board[xstart][ystart] = self.empty

        if len(flipped_pos) == 0:
            return False

        for fp in flipped_pos:
            flipped_pos_board.append(self.num_board(fp))
        return flipped_pos_board

    def get_legal_actions(self, color):
        """
        按照黑白棋的规则获取棋子的合法走法
        """
        direction = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
        op_color = "O" if color == "X" else "X"
        op_color_near_points = []

        board = self._board
        for i in range(8):
            for j in range(8):
                if board[i][j] == op_color:
                    for dx, dy in direction:
                        x, y = i + dx, j + dy
                        if 0 <= x <= 7 and 0 <= y <= 7 and board[x][y] == self.empty and (x, y) not in op_color_near_points:
                            op_color_near_points.append((x, y))
        l = [0, 1, 2, 3, 4, 5, 6, 7]
        for p in op_color_near_points:
            if self._can_fliped(p, color):
                if p[0] in l and p[1] in l:
                    p = self.num_board(p)
                yield p

    def board_num(self, action):
        """
        棋盘坐标转化为数字坐标
        """
        row, col = str(action[1]).upper(), str(action[0]).upper()
        if row in '12345678' and col in 'ABCDEFGH':
            x, y = '12345678'.index(row), 'ABCDEFGH'.index(col)
            return x, y

    def num_board(self, action):
        """
        数字坐标转化为棋盘坐标
        """
        row, col = action
        l = [0, 1, 2, 3, 4, 5, 6, 7]
        if col in l and row in l:
            return chr(ord('A') + col) + str(row + 1)


## 4. 游戏类实现

游戏类负责控制游戏流程，包括玩家切换、落子验证、时间管理等。


In [ ]:
class Game(object):
    def __init__(self, black_player, white_player):
        self.game_init()
        
    def game_init(self):
        self.board = ReversiBoard()
        self.current_player = None
        self.black_player = black_player
        self.white_player = white_player
        self.black_player.color = "X"
        self.white_player.color = "O"

    def switch_player(self, black_player, white_player):
        """
        游戏过程中切换玩家
        """
        if self.current_player is None:
            return black_player
        else:
            if self.current_player == self.black_player:
                return white_player
            else:
                return black_player

    def print_winner(self, winner):
        """
        打印赢家
        """
        print(['黑棋获胜!', '白棋获胜!', '平局'][winner])

    def force_loss(self, is_timeout=False, is_board=False, is_legal=False):
        """
        落子3个不合符规则和超时则结束游戏
        """
        if self.current_player == self.black_player:
            win_color = '白棋 - O'
            loss_color = '黑棋 - X'
            winner = 1
        else:
            win_color = '黑棋 - X'
            loss_color = '白棋 - O'
            winner = 0

        if is_timeout:
            print('\n{} 思考超过 60s, {} 胜'.format(loss_color, win_color))
        if is_legal:
            print('\n{} 落子 3 次不符合规则,故 {} 胜'.format(loss_color, win_color))
        if is_board:
            print('\n{} 擅自改动棋盘判输,故 {} 胜'.format(loss_color, win_color))

        diff = 0
        return winner, diff

    def run(self):
        """
        运行游戏
        """
        total_time = {"X": 0, "O": 0}
        step_time = {"X": 0, "O": 0}
        winner = None
        diff = -1

        print('\n=====开始游戏!=====\n')
        self.board.display(step_time, total_time)
        while True:
            self.current_player = self.switch_player(self.black_player, self.white_player)
            start_time = datetime.datetime.now()
            color = "X" if self.current_player == self.black_player else "O"
            legal_actions = list(self.board.get_legal_actions(color))
            if len(legal_actions) == 0:
                if self.game_over():
                    winner, diff = self.board.get_winner()
                    break
                else:
                    continue

            board = deepcopy(self.board._board)

            try:
                for i in range(0, 3):
                    action = func_timeout(60, self.current_player.get_move, kwargs={'board': self.board})
                    if action == "Q":
                        break
                    if action not in legal_actions:
                        print("你落子不符合规则,请重新落子！")
                        continue
                    else:
                        break
                else:
                    winner, diff = self.force_loss(is_legal=True)
                    break
            except FunctionTimedOut:
                winner, diff = self.force_loss(is_timeout=True)
                break

            end_time = datetime.datetime.now()
            if board != self.board._board:
                winner, diff = self.force_loss(is_board=True)
                break
            if action == "Q":
                winner, diff = self.board.get_winner()
                break

            if action is None:
                continue
            else:
                es_time = (end_time - start_time).seconds
                if es_time > 60:
                    print('\n{} 思考超过 60s'.format(self.current_player))
                    winner, diff = self.force_loss(is_timeout=True)
                    break

                self.board._move(action, color)
                if self.current_player == self.black_player:
                    step_time["X"] = es_time
                    total_time["X"] += es_time
                else:
                    step_time["O"] = es_time
                    total_time["O"] += es_time
                self.board.display(step_time, total_time)

                if self.game_over():
                    winner, diff = self.board.get_winner()
                    break

        print('\n=====游戏结束!=====\n')
        self.board.display(step_time, total_time)
        self.print_winner(winner)

        if winner is not None and diff > -1:
            result = {0: 'black_win', 1: 'white_win', 2: 'draw'}[winner]

    def game_over(self):
        """
        判断游戏是否结束
        """
        b_list = list(self.board.get_legal_actions('X'))
        w_list = list(self.board.get_legal_actions('O'))
        is_over = len(b_list) == 0 and len(w_list) == 0
        return is_over


## 5. 基础函数和树节点类

定义MCTS算法所需的辅助函数和树节点数据结构。


In [ ]:
def oppo(color):
    """
    交换棋手
    """
    if color == 'X':
        return 'O'
    return 'X'
    
class TreeNode():
    """
    蒙特卡洛树节点
    
    属性说明：
    - parent: 父节点
    - w: 该节点获胜次数 (win count)
    - n: 该节点被访问次数 (visit count)
    - color: 当前棋手颜色
    - child: 子节点字典，key为落子动作，value为子节点
    """  
    
    def __init__(self, parent, color):
        self.parent = parent
        self.w = 0  # 胜利次数
        self.n = 0  # 访问次数
        self.color = color
        self.child = dict()  # 存储子节点


## 6. Roxanne策略玩家

Roxanne策略是一种基于位置优先级的启发式策略，用于快速模拟和决策。


In [ ]:
class RoxannePlayer(object):
    """
    Roxanne 策略
    详见 《Analysis of Monte Carlo Techniques in Othello》
    提出者：Canosa, R. Roxanne canosa homepage. https://www.cs.rit.edu/~rlc/
    
    策略说明：
    - roxanne_table中的位置按照优先级从高到低排列
    - 角位（A1, H1, A8, H8）优先级最高，因为角位不会被翻转
    - 次优先考虑中心稳定位置
    - 避免选择靠近角位的危险位置（会给对手占角机会）
    """

    def __init__(self, color):
        """
        Roxanne策略初始化
        :param roxanne_table: 从上到下依次按落子优先级排序
        :param color: 执棋方
        """
        self.roxanne_table = [
            ['A1', 'H1', 'A8', 'H8'],  # 角位：最高优先级
            ['C3', 'F3', 'C6', 'F6'],  # 次稳定位置
            ['C4', 'F4', 'C5', 'F5', 'D3', 'E3', 'D6', 'E6'],  # 中心区域
            ['A3', 'H3', 'A6', 'H6', 'C1', 'F1', 'C8', 'F8'],
            ['A4', 'H4', 'A5', 'H5', 'D1', 'E1', 'D8', 'E8'],
            ['B3', 'G3', 'B6', 'G6', 'C2', 'F2', 'C7', 'F7'],
            ['B4', 'G4', 'B5', 'G5', 'D2', 'E2', 'D7', 'E7'],
            ['B2', 'G2', 'B7', 'G7'],  # 靠近角位的危险位置
            ['A2', 'H2', 'A7', 'H7', 'B1', 'G1', 'B8', 'G8']  # 最低优先级
        ]
        self.color = color

    def roxanne_select(self, board):
        """
        采用Roxanne 策略选择落子策略
        :return: 落子策略
        """
        action_list = list(board.get_legal_actions(self.color))
        if len(action_list) == 0:
            return None
        else:
            # 按优先级顺序遍历位置
            for move_list in self.roxanne_table:
                random.shuffle(move_list)  # 同优先级位置随机选择
                for move in move_list:
                    if move in action_list:
                        return move

    def get_move(self, board):
        """
        采用Roxanne 策略进行搜索
        :return: 落子
        """
        action = self.roxanne_select(board)
        return action


## 7. 静默游戏类

SilentGame类用于MCTS的模拟阶段，不打印棋盘，快速模拟对局。


In [ ]:
class SilentGame(object):
    """
    重构游戏类，模拟下棋过程中，不实时打印棋盘
    用于MCTS的快速模拟阶段
    """
    
    def __init__(self, black_player, white_player, board=ReversiBoard(), current_player=None):
        self.board = deepcopy(board)
        self.current_player = current_player
        self.black_player = black_player
        self.white_player = white_player
        self.black_player.color = "X"
        self.white_player.color = "O"

    def switch_player(self, black_player, white_player):
        """
        游戏过程中切换玩家
        """
        if self.current_player is None:
            return black_player
        else:
            if self.current_player == self.black_player:
                return white_player
            else:
                return black_player

    def run(self):
        """
        运行游戏（快速模拟，无输出）
        """
        while True:
            self.current_player = self.switch_player(self.black_player, self.white_player)
            color = "X" if self.current_player == self.black_player else "O"
            legal_actions = list(self.board.get_legal_actions(color))
            
            if len(legal_actions) == 0:
                if self.game_over():
                    winner, diff = self.board.get_winner()
                    break
                else:
                    continue

            action = self.current_player.get_move(self.board)

            if action is None:
                continue
            else:
                self.board._move(action, color)
                if self.game_over():
                    winner, diff = self.board.get_winner()
                    break

        return winner, diff

    def game_over(self):
        """
        判断游戏是否结束
        """
        b_list = list(self.board.get_legal_actions('X'))
        w_list = list(self.board.get_legal_actions('O'))
        is_over = len(b_list) == 0 and len(w_list) == 0
        return is_over


## 8. MCTS AI玩家 - 核心算法实现

这是本次作业的核心部分，实现了完整的蒙特卡洛树搜索算法，包括：
1. **UCB算法**：用于节点选择，平衡探索与利用
2. **树扩展**：为叶子节点添加所有可能的子节点
3. **快速模拟**：使用Roxanne策略快速模拟到游戏结束
4. **反向传播**：更新搜索路径上所有节点的统计信息


In [ ]:
class AIPlayer(object):
    """
    蒙特卡洛树搜索智能算法
    """
    
    def __init__(self, color, time_limit=2):
        """
        蒙特卡洛树搜索策略初始化
        :param color: 执棋方
        :param time_limit: 蒙特卡洛树搜索每步的搜索时间步长（秒）
        :param tick: 记录开始搜索的时间
        :param sim_black, sim_white: 采用Roxanne策略代替随机策略搜索
        """
        self.time_limit = time_limit
        self.tick = 0
        self.sim_black = RoxannePlayer('X')
        self.sim_white = RoxannePlayer('O')
        self.color = color

    def mcts(self, board):
        """
        蒙特卡洛树搜索主函数
        在时间限制范围内，循环进行选择、扩展、模拟、反向传播四个步骤
        :param board: 当前棋盘状态
        :return: 选择访问次数最多的子节点对应的动作
        """
        root = TreeNode(None, self.color)
        
        # 在时间限制内不断进行MCTS迭代
        while time() - self.tick < self.time_limit - 1:
            sim_board = deepcopy(board)  # 复制棋盘用于模拟
            choice = self.select(root, sim_board)  # 选择阶段
            self.expand(choice, sim_board)  # 扩展阶段
            winner, diff = self.simulate(choice, sim_board)  # 模拟阶段
            
            # 计算反向传播的分数
            back_score = [1, 0, 0.5][winner]  # 黑胜:1, 白胜:0, 平局:0.5
            if choice.color == 'X':
                back_score = 1 - back_score  # 如果当前节点是黑棋，需要反转分数
            
            self.back_prop(choice, back_score)  # 反向传播阶段

        # 选择访问次数最多的子节点
        best_n = -1
        best_move = None
        for k in root.child.keys():
            if root.child[k].n > best_n:
                best_n = root.child[k].n
                best_move = k
        return best_move

    def select(self, node, board):
        """
        【作业任务1：UCB算法实现】
        
        蒙特卡洛树搜索 - 选择阶段
        使用UCB (Upper Confidence Bound) 算法选择最优子节点
        
        UCB公式：UCB(node) = w/n + C * sqrt(ln(N)/n)
        其中：
        - w: 节点胜利次数
        - n: 节点访问次数
        - N: 父节点访问次数
        - C: 探索常数，通常取sqrt(2)，用于平衡探索与利用
        
        算法思想：
        1. 如果节点没有子节点（叶子节点），直接返回
        2. 如果有子节点未被访问过（n=0），优先选择未访问的节点
        3. 否则使用UCB公式计算每个子节点的分数，选择分数最高的
        
        UCB公式的两部分含义：
        - w/n：利用项，倾向于选择胜率高的节点
        - C*sqrt(ln(N)/n)：探索项，倾向于选择访问次数少的节点
        
        :param node: 当前节点
        :param board: 当前棋盘状态
        :return: 递归选择到的叶子节点
        """
        if len(node.child) == 0:
            # 如果当前节点是叶子节点，返回该节点
            return node
        else:
            best_score = -1
            best_move = None
            
            # 遍历所有子节点
            for k in node.child.keys():
                child_node = node.child[k]
                
                if child_node.n == 0:
                    # 优先选择未访问过的节点
                    best_move = k
                    break
                else:
                    # ========== UCB算法核心实现 ==========
                    # UCB = 利用项 + 探索项
                    # 利用项：w/n，表示该节点的平均胜率
                    exploitation = child_node.w / child_node.n
                    
                    # 探索项：C * sqrt(ln(N)/n)
                    # C取sqrt(2)是理论最优值
                    # ln(N)/n 表示父节点访问次数与子节点访问次数的对数比
                    exploration = sqrt(2) * sqrt(log(node.n) / child_node.n)
                    
                    # UCB分数 = 利用项 + 探索项
                    ucb_score = exploitation + exploration
                    # ========================================
                    
                    # 记录最高分数的动作
                    if ucb_score > best_score:
                        best_score = ucb_score
                        best_move = k

            # 在棋盘上执行选中的动作
            board._move(best_move, node.color)
            # 递归地对选中的子节点继续进行选择
            return self.select(node.child[best_move], board)

    def expand(self, node, board):
        """
        蒙特卡洛树搜索 - 扩展阶段
        为当前节点添加所有可能的子节点
        
        :param node: 当前节点
        :param board: 当前棋盘状态
        """
        # 获取所有合法动作
        for move in board.get_legal_actions(node.color):
            # 为每个合法动作创建一个子节点
            # 子节点的颜色是对手的颜色
            node.child[move] = TreeNode(node, oppo(node.color))

    def simulate(self, node, board):
        """
        【作业任务2：改进的模拟策略】
        
        蒙特卡洛树搜索 - 模拟阶段
        使用Roxanne启发式策略代替完全随机策略进行快速模拟
        
        改进说明：
        - 原始MCTS使用随机策略模拟，效率低且结果随机性大
        - Roxanne策略基于黑白棋领域知识，优先选择好位置
        - 这种启发式策略可以显著提高模拟质量和搜索效率
        
        :param node: 当前节点
        :param board: 当前棋盘状态
        :return: (winner, diff) 胜者和棋子差
        """
        # 根据当前节点的颜色确定下一步棋手
        if node.color == 'O':
            current_player = self.sim_black
        else:
            current_player = self.sim_white
        
        # 使用SilentGame进行快速模拟
        # 双方都使用Roxanne策略进行决策
        sim_game = SilentGame(self.sim_black, self.sim_white, board, current_player)
        
        # 运行模拟直到游戏结束
        return sim_game.run()

    def back_prop(self, node, score):
        """
        蒙特卡洛树搜索 - 反向传播阶段
        回溯更新模拟路径中的所有节点的统计信息
        
        :param node: 当前节点
        :param score: 本次模拟的得分（从当前节点视角）
        """
        # 更新节点的访问次数
        node.n += 1
        # 更新节点的胜利次数
        node.w += score
        
        # 如果有父节点，递归向上传播
        if node.parent is not None:
            # 注意：父节点的得分是 1-score，因为父节点是对手
            self.back_prop(node.parent, 1 - score)

    def get_move(self, board):
        """
        AI玩家的主入口函数
        使用MCTS算法选择最佳落子位置
        
        :param board: 当前棋盘状态
        :return: 选择的动作
        """
        self.tick = time()  # 记录开始时间
        action = self.mcts(deepcopy(board))  # 运行MCTS算法
        return action


In [ ]:
class HumanPlayer:
    """
    人类玩家
    """

    def __init__(self, color):
        """
        玩家初始化
        :param color: 下棋方，'X' - 黑棋，'O' - 白棋
        """
        self.color = color

    def get_move(self, board):
        """
        根据当前棋盘输入人类合法落子位置
        :param board: 棋盘
        :return: 人类下棋落子位置
        """
        if self.color == "X":
            player = "黑棋"
        else:
            player = "白棋"

        while True:
            action = input(
                "请'{}-{}'方输入一个合法的坐标(e.g. 'D3'，若不想进行，请务必输入'Q'结束游戏。): ".format(player, self.color))

            if action == "Q" or action == 'q':
                return "Q"
            else:
                row, col = action[1].upper(), action[0].upper()

                if row in '12345678' and col in 'ABCDEFGH':
                    if action in board.get_legal_actions(self.color):
                        return action
                else:
                    print("你的输入不合法，请重新输入!")


## 10. 测试代码

测试AI vs AI的对局。


In [ ]:
# 创建两个AI玩家进行对弈测试
# 黑棋AI，每步思考时间2秒
black_player = AIPlayer("X", time_limit=2)

# 白棋AI，每步思考时间2秒  
white_player = AIPlayer("O", time_limit=2)

# 初始化游戏
game = Game(black_player, white_player)

# 开始游戏
# game.run()  # 取消注释以运行游戏

print("游戏代码已准备就绪！")
print("取消注释 game.run() 即可开始AI对弈。")


---

## 算法实现总结

本notebook实现了完整的蒙特卡洛树搜索（MCTS）算法用于黑白棋AI，主要包含两个核心任务的实现：

### 任务一：UCB算法实现

**UCB公式：** $UCB(v_i) = \frac{w_i}{n_i} + \sqrt{2} \times \sqrt{\frac{\ln N}{n_i}}$

实现要点：
- **利用项** $\frac{w_i}{n_i}$：选择胜率高的节点
- **探索项** $\sqrt{2} \times \sqrt{\frac{\ln N}{n_i}}$：探索访问次数少的节点
- **未访问节点优先**：对于 $n_i = 0$ 的节点直接选择
- **递归选择**：从根节点递归选择到叶子节点

### 任务二：Roxanne启发式策略

实现要点：
- **位置优先级表**：基于黑白棋理论，角位最高，X-square最低
- **快速模拟**：替代完全随机策略，提高模拟质量
- **领域知识融合**：结合黑白棋的稳定性、活动性等概念
- **同优先级随机**：增加策略多样性

### MCTS四个阶段

1. **选择（Selection）**：使用UCB算法选择最优路径
2. **扩展（Expansion）**：为叶子节点添加所有合法动作的子节点
3. **模拟（Simulation）**：使用Roxanne策略快速模拟到游戏结束
4. **反向传播（Backpropagation）**：更新路径上所有节点的统计信息

### 算法特点

- ✅ **理论完备**：UCB算法具有渐进最优性保证
- ✅ **实践高效**：Roxanne策略显著提升模拟质量
- ✅ **代码清晰**：详细注释，易于理解和扩展
- ✅ **时间可控**：准确的时间管理，避免超时

### 详细文档

完整的算法实现报告请查看：`算法实现报告.md`

报告包含：
- 详细的理论分析和公式推导
- UCB算法的数值示例和行为分析
- Roxanne策略的黑白棋领域知识解释
- 复杂度分析和优化方向
- 实验设计和结果讨论

---

**实现日期**：2025年11月28日  
**算法版本**：MCTS + UCB + Roxanne策略
